In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
cd /kaggle/input/netflix-prize-data/

In [ ]:
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
#from surprise import Reader, Dataset, SVD 
sns.set_style("ticks")
from pandas import pivot_table, pivot
from surprise import Reader
from surprise.prediction_algorithms.matrix_factorization import SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate
from scipy.linalg import svdvals

In [ ]:
import surprise
from surprise import accuracy
from surprise.model_selection import train_test_split

In [ ]:
%matplotlib inline

My colleagues tell me that I have a deep understanding of the math of Data Science, Linear Algebra, Calculus and Probability Theory, **but** my code game is weak as I am still very new to coding.  This project took me considerably longer than it should have; and, I had to reverse engineer other people's code just the littlest bit to finish.  The reader will notice that my code is so badly malfunctioning as to be barely be working.

In [ ]:
movie_titles = pd.read_csv('movie_titles.csv', encoding = "ISO-8859-1", header = None, names = ['Movie Id', 'Year', 'Title'])
movie_titles.sort_values(by=['Title'], axis=0, ascending=True, inplace=True)
#movie_titles.set_index('Title', inplace = True)
combined_1 = pd.read_csv('combined_data_1.txt', header = None, names = ['Cust Id', 'Rating'], usecols = [0,1])
combined_2 = pd.read_csv('combined_data_2.txt', header = None, names = ['Cust Id', 'Rating'], usecols = [0,1])
combined_3 = pd.read_csv('combined_data_3.txt', header = None, names = ['Cust Id', 'Rating'], usecols = [0,1])
combined_4 = pd.read_csv('combined_data_4.txt', header = None, names = ['Cust Id', 'Rating'], usecols = [0,1])
probe = pd.read_csv('probe.txt', header = None, names = ['Cust Id'], usecols = [0])
qualifying = pd.read_csv('qualifying.txt', header = None, names = ['Cust Id', 'Release Date'], usecols = [0,1])





**Treat the Data**

In [ ]:
titles = movie_titles
cmbnd1 = combined_1[:10000]
cmbnd2 = combined_2[:10000]
cmbnd3 = combined_3[:10000]
cmbnd4 = combined_4[:10000]
qual = qualifying[:10000]
test = probe[:10000]

In [ ]:
opt_set = [cmbnd1, cmbnd2, cmbnd3, cmbnd4]


In [ ]:
for i in opt_set:   
    i['Movie Id'] = i['Cust Id'].str.extract('(\d+:)')
    i['Movie Id'].ffill(axis=0, inplace=True, limit=None, downcast=np.int64)
    i['Movie Id'].replace('[:]', '', regex=True, inplace=True)
    i.dropna(axis=0, inplace=True)
    #i.reset_index(inplace=True)
    i = i.astype(int)
    i = i.drop_duplicates(subset= ['Cust Id'], keep='last')
    i = i.drop_duplicates(subset= ['Movie Id'], keep='last')
    #i = pd.pivot_table(i, index='Movie Id', columns= 'Cust Id', values='Rating', aggfunc=np.mean, fill_value=0.00001)
    #i = np.asanyarray(i, dtype=np.int64)
    #This seems like a hot ticket right here
        #i = i.stack(list(range(i.columns.nlevels())))
        #'int' object is not callable
    print(i)
    #print(i.show())

In [ ]:
cmbnd = cmbnd1.append(cmbnd2, ignore_index=False) 
cmbnd = cmbnd.append(cmbnd3, ignore_index=False)
cmbnd = cmbnd.append(cmbnd4, ignore_index=False)

In [ ]:
matrix = csr_matrix(cmbnd.astype(float))


**Clearly State Priors**

Bayes enters here.  The Users may only enter movies if she rates them a '5'.  In the original, the team minimized the L2 Norm.  The used Bayes to update the Users' new recommendations.

In [ ]:
movies_i_love = [6303, 2945, 14707, 17655, 1288, 9333, 7756, 8575]


In [ ]:
print('The Movie Id of one my favs is')
key = int(input())


In [ ]:
movies_i_love.append(key)

In [ ]:
loved_movies = movie_titles.copy()
loved_movies = loved_movies.loc[movie_titles['Movie Id'].isin(movies_i_love)]
loved_movies['Rating'] = 5.0
loved_movies.drop('Title', axis=1, inplace=True)
loved_movies['Rating'].astype(float)
loved_movies.set_index(loved_movies['Movie Id'], inplace=True)

**Visualize the Data**

In [ ]:
combined_1.plot.hist(cumulative=True)

In [ ]:
combined_2.plot.hist(cumulative=True)


In [ ]:
combined_3.plot.hist(cumulative=True)


In [ ]:
combined_4.plot.hist(cumulative=True)


In [ ]:
combined_5.plot.hist(cumulative=True)


**Fator the Matrix with SVD**

In [ ]:
from surprise.model_selection import KFold

In [ ]:
reader = Reader()
svd = SVD()

In [ ]:
data = Dataset.load_from_df(cmbnd, reader)
#data is my data_set now
trainset = data.build_full_trainset()


In [ ]:
svd.fit(trainset)

In [ ]:
kf = KFold(n_splits=5)

In [ ]:
for trainset, testset in kf.split(data):
    svd.fit(trainset)
    predictions = svd.test(testset)

This is where I am getting stuck... I need to format predictions correctly...

In [ ]:
trainset, testset = train_test_split(matrix, test_size=.25)

In [ ]:
accuracy.rmse(predictions)


In [ ]:
cross_validate(svd, matrix, measures=['RMSE', 'MAE'], cv=5, verbose=True)


**Predict with Pearson's Correlation Coefficient**

In [ ]:
def recommend(movie_title, min_count):
    print("For movie ({})".format(movie_title))
    print("- Top 10 movies recommended based on Pearsons'R correlation - ")
    i = int(df_title.index[df_title['Name'] == movie_title][0])
    target = df_p[i]
    similar_to_target = df_p.corrwith(target)
    corr_target = pd.DataFrame(similar_to_target, columns = ['PearsonR'])
    corr_target.dropna(inplace = True)
    corr_target = corr_target.sort_values('PearsonR', ascending = False)
    corr_target.index = corr_target.index.map(int)
    corr_target = corr_target.join(df_title).join(df_movie_summary)[['PearsonR', 'Name', 'count', 'mean']]
    print(corr_target[corr_target['count']>min_count][:10].to_string(index=False))

The previous works but I have to iron_it_out...